In [ ]:
import numpy as np
from skimage import filters, morphology
import matplotlib.pyplot as plt
import math
import random
from PIL import Image

In [ ]:
img = Image.open('sample.jpg')
img.load()
data = np.asarray(img, dtype=np.int32)

## Salt and pepper noise

In [ ]:
sp_data = np.copy(data)
sp_data.shape

In [ ]:
n = 4000

for _ in range(n):
    i = random.randint(0, sp_data.shape[0] - 1)
    j = random.randint(0, sp_data.shape[1] - 1)

    flag = bool(random.getrandbits(1))
    
    for _ in range(3):
        if flag:
            sp_data[i][j][_] = 255
        else:
            sp_data[i][j][_] = 0

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(sp_data, interpolation='nearest')

## Median filter (skimage)

In [ ]:
mf_data = np.copy(np.rollaxis(data, 2))
mf_data.shape

In [ ]:
for _ in range(mf_data.shape[0]):
    mf_data[_] = filters.median(mf_data[_], morphology.square(9))
mf_data = np.rollaxis(mf_data, 2)
mf_data = np.rollaxis(mf_data, 2)
mf_data.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(mf_data, interpolation='nearest')

## Box filter (skimage)

In [ ]:
bf_data = np.copy(np.rollaxis(data, 2))
bf_data.shape

In [ ]:
for _ in range(bf_data.shape[0]):
    bf_data[_] = filters.rank.mean(bf_data[_], morphology.square(9))
bf_data = np.rollaxis(bf_data, 2)
bf_data = np.rollaxis(bf_data, 2)
bf_data.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(bf_data, interpolation='nearest')

## Gaussian filter (skimage)

In [ ]:
gf_data = np.copy(data)
gf_data.shape

In [ ]:
gf_data = filters.gaussian(gf_data, sigma=1, multichannel=True, preserve_range=True).astype(np.int32)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(gf_data, interpolation='nearest')

## Median filter (handcrafted)

In [ ]:
mf2_data = np.copy(np.rollaxis(data, 2))
mf2_data.shape

In [ ]:
def median_filter(data, wsize):
    h = data.shape[0]
    w = data.shape[1]
    d = math.floor(wsize/2)
    res = np.empty_like(data)
    for i in range(h):
        for j in range(w):
            tmp = data[max(0,i-d):min(h,i+d), max(0,j-d):min(w,j+d)]
            res[i][j] = np.median(tmp)
    return res

In [ ]:
for _ in range(mf2_data.shape[0]):
    mf2_data[_] = median_filter(mf2_data[_], 9)
mf2_data = np.rollaxis(mf2_data, 2)
mf2_data = np.rollaxis(mf2_data, 2)
mf2_data.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(mf2_data, interpolation='nearest')

## Box filter (handcrafted)

In [ ]:
bf2_data = np.copy(np.rollaxis(data, 2))
bf2_data.shape

In [ ]:
def box_filter(data, wsize):
    h = data.shape[0]
    w = data.shape[1]
    d = math.floor(wsize/2)
    res = np.empty_like(data)
    for i in range(h):
        for j in range(w):
            tmp = data[max(0,i-d):min(h,i+d), max(0,j-d):min(w,j+d)]
            res[i][j] = np.mean(tmp)
    return res

In [ ]:
for _ in range(bf2_data.shape[0]):
    bf2_data[_] = box_filter(bf2_data[_], 9)
bf2_data = np.rollaxis(bf2_data, 2)
bf2_data = np.rollaxis(bf2_data, 2)
bf2_data.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(bf2_data, interpolation='nearest')

## Comparison of (skimage) and (handcrafted) filters

In [ ]:
total_pixels = mf_data.shape[0] * mf_data.shape[1] * mf_data.shape[2]
print('Similarity of (handcrafted) and (skimage) versions: ')
print('Median filter: ' + '{:.3}'.format(np.sum(mf_data == mf2_data) / total_pixels))
print('Box filter: ' + '{:.3}'.format(np.sum(bf_data == bf2_data) / total_pixels))

## Difference of gaussians

In [ ]:
dog1_data = np.copy(data)
dog2_data = np.copy(data)

In [ ]:
g1 = filters.gaussian(dog1_data, sigma=2, multichannel=True, preserve_range=True).astype(np.int32)
g2 = filters.gaussian(dog2_data, sigma=5, multichannel=True, preserve_range=True).astype(np.int32)
dog_data = g1 - g2

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(dog_data, interpolation='nearest')

## Difference of box filters

In [ ]:
dob1_data = np.copy(np.rollaxis(data, 2))
dob2_data = np.copy(np.rollaxis(data, 2))

In [ ]:
for _ in range(dob1_data.shape[0]):
    dob1_data[_] = filters.rank.mean(dob1_data[_], morphology.square(19))
    dob2_data[_] = filters.rank.mean(dob2_data[_], morphology.square(43))
    
dob1_data = np.rollaxis(dob1_data, 2)
dob1_data = np.rollaxis(dob1_data, 2)
dob2_data = np.rollaxis(dob2_data, 2)
dob2_data = np.rollaxis(dob2_data, 2)

dob_data = dob1_data - dob2_data

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(dob_data, interpolation='nearest')